# It's only my choice
 
This notebook is a collection of few snippets from other notebooks from TPS March 2021 which I found to be unique in some way and/or eye-catcher in terms of their presentation. Ofcourse there are many great notebooks interms of content (new ideas/code/lb-score), completeness of analysis, other tools such as AutoML, AutoViz and so on. Here I chose few sample visuaization from uniquess and appeal point of view. There can also be other notebooks with similar plots the choice here is purely by chance.
 
**Note**: I do not claim to have reviewed/visited all the notebooks from this months tabular playground series. So it is possible that I may have missed some peculiar plots that could be in this list. Feel free to point them out in the comments. I will be more that happy to discover and learn more tricks and vizualizaton techniques. 
 
Thanks to @subinium, @joshuaswords, @ruchi798 @siavrez and @ksvmuralidhar for sharing their notebooks. They wrote the code snippets for the plots presented below.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import networkx as nx
import category_encoders as ce

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

# 1. Subin Ans' (UMAP Embedding space)

### [[TPS-March] EDA + Dimension Reduction](https://www.kaggle.com/subinium/tps-march-eda-dimension-reduction)

In [ ]:
from umap import UMAP

# Dimension Reduction
train_sub = train.sample(10000)
target = train_sub['target'] 
data_sub = train_sub.drop([col for col in train.columns if 'cat' in col], axis=1)

umap = UMAP(random_state=0)
train_umap = umap.fit_transform(data_sub, target)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 12))

ax[0][0].scatter(train_umap[target==1][:,0], train_umap[target==1][:,1], c='red', alpha=0.25, label='Target:1')
ax[0][0].scatter(train_umap[target==0][:,0], train_umap[target==0][:,1], c='blue', alpha=0.25, label='Target:0')


ax[0][1].set_visible(False)

ax[1][0].scatter(train_umap[target==1][:,0], train_umap[target==1][:,1], c='red', alpha=0.25, label='Target:1')
ax[1][1].scatter(train_umap[target==0][:,0], train_umap[target==0][:,1], c='blue', alpha=0.25, label='Target:0')

for i in range(2):
    for j in range(2):
        ax[i][j].set_xticks([])
        ax[i][j].set_yticks([])
        for s in ["top","right","left", 'bottom']:
            ax[i][j].spines[s].set_visible(False)

        ax[i][j].legend()

# Text Part
fig.text(0.97, 1, '[UMAP] Embedding Space', fontweight='bold', fontfamily='serif', fontsize=20, ha='right')   

fig.text(0.97, 0.94, '''
In dimension reduction, 
you can see that the data with target=0 are gathered in the center.
Performance can be expected when used as a feature.

''', 
         fontweight='light', fontfamily='serif', fontsize=12, va='top', ha='right')           
        
plt.tight_layout()
plt.show()


# 2. Joshua Swords' (Correlation plot)

### [TPS EDA & Model [March 2020]](https://www.kaggle.com/joshuaswords/tps-eda-model-march-2020)

In [ ]:
train_df = train.copy()
test_df = test.copy()

feature_cols = train_df.drop(['id', 'target'], axis=1).columns

numerical_columns = train_df[feature_cols].select_dtypes(include=['int64','float64']).columns
categorical_columns = train_df[feature_cols].select_dtypes(exclude=['int64','float64']).columns


background_color = "#f6f5f5"

fig = plt.figure(figsize=(18, 8), facecolor=background_color)
gs = fig.add_gridspec(1, 2)
gs.update(wspace=-0.36, hspace=0.27)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
colors = ["#fbfbfb", "lightgray","#0e4f66"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax0.set_facecolor(background_color)
ax0.text(0, 0, 'Train', fontsize=20, fontweight='bold', fontfamily='serif',color='lightgray')

ax1.set_facecolor(background_color)
ax1.text(9.5, 11, 'Test', fontsize=20, fontweight='bold', fontfamily='serif',color='lightgray')


fig.text(0.5,0.5,'Correlation of Features\nFor Train & Test\nDatasets', fontsize=20, fontweight='bold', fontfamily='serif',va='center',ha='center')

corr = train_df[numerical_columns].corr().abs()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
sns.heatmap(corr, ax=ax0, vmin=-1, vmax=1, annot=True, square=True, mask=mask,
            cbar_kws={"orientation": "horizontal"}, cbar=False, cmap=colormap, fmt='.1g',linewidth=3,linecolor=background_color)


corr = test_df[numerical_columns].corr().abs()
mask = np.tril(corr)
sns.heatmap(corr, ax=ax1, vmin=-1, vmax=1, annot=True, square=True, mask=mask,
            cbar_kws={"orientation": "horizontal"}, cbar=False, cmap=colormap, fmt='.1g',linewidth=3,linecolor=background_color)
ax1.xaxis.tick_top()
ax1.yaxis.tick_right()

plt.show()

# 3. Ruchi Bhatia's (Features Network)

### [TPS March 2021: EDA+RAPIDS 🏃‍♀️](https://www.kaggle.com/ruchi798/tps-march-2021-eda-rapids)

In [ ]:
corr=train_df.corr()
indices = corr.index.values
cor_matrix = np.asmatrix(corr)
G = nx.from_numpy_matrix(cor_matrix)
G = nx.relabel_nodes(G,lambda x: indices[x])
G.edges(data=True)

In [ ]:
def corr_network(G, corr_direction, min_correlation):
    H = G.copy()

    for s1, s2, weight in G.edges(data=True):       
        if corr_direction == "positive":
            if weight["weight"] < 0 or weight["weight"] < min_correlation:
                H.remove_edge(s1, s2)
        else:
            if weight["weight"] >= 0 or weight["weight"] > min_correlation:
                H.remove_edge(s1, s2)
                
    edges,weights = zip(*nx.get_edge_attributes(H,'weight').items())
    
    weights = tuple([(1+abs(x))**2 for x in weights])
   
    d = dict(nx.degree(H))
    nodelist=d.keys()
    node_sizes=d.values()
    
    positions=nx.circular_layout(H)
    
    plt.figure(figsize=(15,15))

    nx.draw_networkx_nodes(H,positions,node_color='#d100d1',nodelist=nodelist,
                       node_size=tuple([x**3 for x in node_sizes]),alpha=0.8)

    nx.draw_networkx_labels(H, positions, font_size=8)

    if corr_direction == "positive":
        edge_colour = plt.cm.cool 
    else:
        edge_colour = plt.cm.Wistia
        
    nx.draw_networkx_edges(H, positions, edgelist=edges,style='solid',
                          width=weights, edge_color = weights, edge_cmap = edge_colour,
                          edge_vmin = min(weights), edge_vmax=max(weights))
    plt.axis('off')
    plt.show() 

In [ ]:
corr_network(G, corr_direction="positive", min_correlation = 0)

# 4. Siavash's  (UMAP-TARGET)
### [EDA-Starter](https://www.kaggle.com/siavrez/eda-starter)

In [ ]:
dense_features = [f for f in train.columns if 'cont' in f]
low_cardinality_sparse_features = [f for f in train.columns if 'cat' in f and train[f].nunique()<17]
high_cardinality_sparse_features = [f for f in train.columns if 'cat' in f and train[f].nunique()>=17]
sparse_features = [f for f in train.columns if 'cat' in f]
target = ['target']

In [ ]:
benc_features = []
for col in sparse_features:
    benc = ce.BinaryEncoder(cols=col, drop_invariant=True)
    benc.fit(train[col])
    train_be = benc.transform(train[col])
    be_features = train_be.columns.tolist()
    train[be_features] = train_be.values
    benc_features.extend(be_features)

In [ ]:
features = benc_features + dense_features
train_ = train[features + target].sample(frac=0.05, random_state=1000)

In [ ]:
mapper = UMAP(n_neighbors=20, n_components=2, n_epochs=500, learning_rate=0.5, random_state=1000).fit(train_[features])
umap_embed = mapper.transform(train_[features])
train_['umap_1'] = umap_embed[:,0]
train_['umap_2'] = umap_embed[:,1]

In [ ]:
cmap = plt.cm.get_cmap("flag")
plt.figure(figsize=(15,15))
plt.scatter(train_.umap_1, train_.umap_2, c=train_.target, s=10, cmap=cmap)
plt.legend()
plt.title('UMAP-TARGET')
plt.show()

# 5. KSV Muralidhar's (Numeric Variables vs Target)
### [EDA+FE with MI+XGB+OPTCV-TPS-Mar 2021](https://www.kaggle.com/ksvmuralidhar/eda-fe-with-mi-xgb-optcv-tps-mar-2021)

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
data.drop(columns='id',inplace=True)
categorical_variables = list(data.select_dtypes('object').columns)
numeric_variables = list(data.select_dtypes('float64').columns)
target = list(data.select_dtypes('int64').columns)[0]


r = c = 0
fig,ax = plt.subplots(6,2,figsize=(14,35))
for n,i in enumerate(numerical_columns):
    med = train_df[[i,'target']].groupby('target').median().copy()
    sns.violinplot(x=target,y=i,data=data,ax=ax[r,c],palette=["gray","lightgray"])
    med.plot(ax=ax[r,c],color='black',linewidth=3,linestyle="--",legend=False)
    for x,y in zip(list(med.index),med[i]):
        ax[r,c].text(x=x+0.05,y=y+0.01,s=np.round(y,2),fontsize=10,color='white',backgroundcolor='black')
    ax[r,c].set_title(i.upper()+" by "+"TARGET")
    c+=1
    if (n+1)%2==0:
        r+=1
        c=0
ax[r,c].axis("off")
plt.show()